# Table of Contents
* [Introduction](#introduction)
* [Imports](#imports)
    - [Standard](#standard)
    - [Matplotlib](#matplotlib)
    - [Import OS and Get/Make Directories](#import-os)
    - [Tensorflow & Keras](#tensorflow)
    - [Other Tools](#other-tools)
* [Generate Data](#generate-data)
    - [Load Pexel Images](#load-pexel-images)
        -[search_terms](#search-terms)
        -[Scrape Image from Online](#scrape-images)
    - [Add Watermarks to Images](#add-watermarks)
* [Make cycleGAN Model](#make-cycle-gan)
    - [Data](#data)
    - [Models](#models)
    - [Loss Functions](#loss-functions)
    - [Optimisers](#optimisers)
    - [Checkpoints](#checkpoints)
    - [Training](#training)
    - [Save Models](#save-models)

# Introduction  <a id="introduction"></a>

Before training, the ensemble model is optimised by removing watermarks from images in both the train and test sets. To do this, a cycleGAN is developed, which is a combination of two GANs: hence there are two "generators" and two "discriminators".  The first generator, G_w, makes watermarked images from normal images. The second generator model, G_n, generates normal images from watermarked images. After training the cycleGAN, this generator model, G_n, is used to remove watermarks. Not only does this notebook develop the cycleGAN, but it also scrapes images from online for training the cycleGAN. 

# Imports <a id="imports"></a>

## Standard <a id="standard"></a>

In [ ]:
import numpy  as np
import pandas as pd

## Matplotlib <a id="matplotlib"></a>

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
plt.rcParams["font.family"] = "Times New Roman"

## Import OS and Get/Make Directories <a id="import-os"></a>

In [ ]:
import os 
from pathlib import Path
cwd = os.path.abspath(os.getcwd())
cwd

In [ ]:
fig_dir = Path("/".join(cwd.split("/"))) / "figures"
if fig_dir.exists() == False:
    os.mkdir(fig_dir)

fig_dir

In [ ]:
data_dir = Path("/".join(cwd.split("/"))) / "data"
if data_dir.exists() == False:
    os.mkdir(data_dir)

data_dir

In [ ]:
pexel_img_dir = data_dir / "pexel_img_dir" 
if pexel_img_dir.exists() == False:
    os.mkdir(pexel_img_dir)

pexel_img_dir

In [ ]:
watermark_pexel_img_dir = data_dir / "watermark_pexel_img_dir"
if watermark_pexel_img_dir.exists() == False:
    os.mkdir(watermark_pexel_img_dir)

watermark_pexel_img_dir

In [ ]:
font_dir = data_dir / "fonts"
if font_dir.exists() == False:
    os.mkdir(font_dir)

font_dir

In [ ]:
model_dir = Path(cwd) / "models"
if model_dir.exists() == False:
    os.mkdir(model_dir)

model_dir

## Tensorflow & Keras <a id="tensorflow"></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow_examples.models.pix2pix import pix2pix

## Other Tools <a id="other-tools"></a>

In [ ]:
import PIL
import requests
import shutil
import string
import time
import cv2 
from pexels_api import API 
from tqdm import tqdm
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from IPython.display import clear_output

# Generate Data <a id="generate-data"></a>

## Load Pexel Images <a id="load-pexel-images"></a>

The first step towards training any model is to collect the training data. For this cycleGAN, stock-free images are scraped from Pexels with an [API Key](https://www.pexels.com/api/new/). 

### search_terms <a id="search-terms"></a>

Below is the list of search terms used to search for stock-free images. While most search terms are related to food items because of the main objective of the Kaggle Project, some few terms are unrelated, like "kitten" and "people", to confuse and challenge the model during training.

In [ ]:
search_terms = [
  "kitten",
  "dog",
  "food",
  "cake",
  "pasta",
  "people",
  "steak",
  "cooked chicken",
  "chicken wings",
  "crab food",
  "seafood",
  "oyster",
  "donut",
  "burger",
  "pizza",
  "egg",
  "avocado",
  "bread",
  "salad",
  "sandwich",
  "fries",
  "butter",
  "ham",
  "sausage",
  "bacon",
  "dessert",
  "rice",
  "lasagna",
  "green peas",
]

### Scrape Images from Online <a id="scrape-images"></a>

In [ ]:
pexel_folder_dir = pexel_img_dir / "pexel_images"
if pexel_folder_dir.exists() == False:
    os.mkdir(pexel_folder_dir)

PEXELS_API_KEY = "563492ad6f91700001000001f8f1cdfad4e84affa3d8bb8ea5312020"

api = API(PEXELS_API_KEY)

for i, search_term in enumerate(tqdm(search_terms)):
    print("search_term: ", search_term)
    search = api.search(search_term, page=1, results_per_page=40)
    photos = api.get_entries()
    for j,photo in enumerate(photos):
        img_url = photo.medium
        filename = str(search_term + "_" + str(j) + ".jpg")
        r = requests.get(img_url, stream=True)
        if r.status_code == 200:
            with open(pexel_folder_dir / filename, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

## Add Watermarks to Images <a id="add-watermarks"></a>

The next set in making the dataset for training is copy the collection of pexel images. With this copy, each image is watermarked. The result is that for every unmarked image, there is also an duplicate with watermarks, and vice-versa. While a cycleGAN does not need pairing a of images to train, it useful to still have this transition to stay organised and evaluate the performance of the cycleGAN. 

Some of the code in the block below is implemented from this [blog](https://rickwierenga.com/blog/machine%20learning/GanWatermark.html).

In [ ]:
fonts = os.listdir(font_dir)
if ".DS_Store" in fonts:
    fonts.remove(".DS_Store")

pexel_folder_dir = pexel_img_dir / "pexel_images"
pexel_images = os.listdir(pexel_folder_dir)
if ".DS_Store" in pexel_images:
    pexel_images.remove(".DS_Store")

watermark_folder_dir = watermark_pexel_img_dir / "watermark_pexel_images"
if watermark_folder_dir.exists() == False:
    os.mkdir(watermark_folder_dir)

curr_dir = None

for im in tqdm(pexel_images):
    img = Image.open(pexel_img_dir / im).convert("RGB")
    width, height, _ = np.array(img).shape
    d = PIL.ImageDraw.Draw(img)
    for i in range(np.random.randint(5, 50)):
        font_file = np.random.choice(fonts)
        fnt = PIL.ImageFont.truetype(str(font_dir / font_file), size=np.random.randint(20, 40))
        fnt.size = np.random.randint(40, 125)
        font_width = np.random.random() * width 
        font_height = np.random.random() * height
        d.text(
            (font_width, font_height), 
            ''.join([np.random.choice(list(string.digits + string.ascii_letters)) for x in range(20)]), 
            fill=(np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255)), 
            font=fnt,
        )
        img.save(watermark_pexel_img_dir / str("watermark_" + im))
        curr_dir = watermark_folder_img_dir / str("watermark_" + im)

curr_dir

# Make cycleGAN Model <a id="make-cycle-gan"></a>

Most of the code below is implemented from the [tensorflow tutorial on cycleGAN](https://www.tensorflow.org/tutorials/generative/cyclegan). 

## Data <a id="data"></a>

In [ ]:
batch_size = 1
img_height = 256
img_width = 256

train_normal = tf.keras.preprocessing.image_dataset_from_directory(
  pexel_img_dir,
  validation_split=0.2,
  subset="training",
  color_mode="rgb",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

train_normal

In [ ]:
batch_size = 1
img_height = 256
img_width = 256

test_normal = tf.keras.preprocessing.image_dataset_from_directory(
  pexel_img_dir,
  validation_split=0.2,
  subset="validation",
  color_mode="rgb",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

test_normal

In [ ]:
batch_size = 1
img_height = 256
img_width = 256

train_watermark = tf.keras.preprocessing.image_dataset_from_directory(
  watermark_pexel_img_dir,
  validation_split=0.2,
  subset="training",
  color_mode="rgb",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

train_watermark

In [ ]:
batch_size = 1
img_height = 256
img_width = 256

test_watermark = tf.keras.preprocessing.image_dataset_from_directory(
  watermark_pexel_img_dir,
  validation_split=0.2,
  subset="validation",
  color_mode="rgb",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

test_watermark

In [ ]:
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def preprocess_image_train(image, label):
    image = normalize(image)
    return image

def preprocess_image_test(image, label):
    image = normalize(image)
    return image

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = 1000

train_normal = train_normal.map(
  preprocess_image_train,
  num_parallel_calls=AUTOTUNE
).cache().shuffle(BUFFER_SIZE).batch(1)

test_normal = test_normal.map(
  preprocess_image_test, 
  num_parallel_calls=AUTOTUNE
).cache().shuffle(BUFFER_SIZE).batch(1)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = 1000

train_watermark = train_watermark.map(
    preprocess_image_train, 
    num_parallel_calls=AUTOTUNE
).cache().shuffle(BUFFER_SIZE).batch(1)

test_watermark = test_watermark.map(
    preprocess_image_test,
    num_parallel_calls=AUTOTUNE
).cache().shuffle(BUFFER_SIZE).batch(1)

In [ ]:
sample_normal = next(iter(train_normal))
sample_watermark = next(iter(train_watermark))

## Models <a id="models"></a>

In [ ]:
OUTPUT_CHANNELS = 3

generator_w = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_n = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_n = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_w = pix2pix.discriminator(norm_type='instancenorm', target=False)

## Loss Functions <a id="loss-functions"></a>

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real, generated):
    real_loss = loss_obj(tf.ones_like(real), real)
    generated_loss = loss_obj(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss * 0.5


def generator_loss(generated):
    return loss_obj(tf.ones_like(generated), generated)

In [ ]:
LAMBDA = 10

def calc_cycle_loss(real_image, cycled_image):
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return LAMBDA * loss1

def identity_loss(real_image, same_image):
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return LAMBDA * 0.5 * loss

## Optimisers <a id="optimisers"></a>

In [ ]:
generator_w_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_n_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_n_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_w_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Checkpoints <a id="checkpoints"></a>

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_w,
                           generator_f=generator_n,
                           discriminator_x=discriminator_n,
                           discriminator_y=discriminator_w,
                           generator_g_optimizer=generator_w_optimizer,
                           generator_f_optimizer=generator_n_optimizer,
                           discriminator_x_optimizer=discriminator_n_optimizer,
                           discriminator_y_optimizer=discriminator_w_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

## Training <a id="training"></a>

In [ ]:
def generate_images(model, test_input):
    prediction = model(test_input)
    plt.figure(figsize=(12, 12))
    display_list = [test_input[0], prediction[0]]
    title = ['Input Image', 'Predicted Image']
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(60, 46))
    plt.subplots_adjust(wspace=0.3)
    for i in range(2):
        ax[i].imshow(display_list[i] * 0.5 + 0.5) 
        ax[i].set_title(title[i])
        ax[i].axis('off')
    plt.savefig(fig_dir / "curr_removal.png", bbox_inches="tight")

In [ ]:
@tf.function
def train_step(real_n, real_w):
    # persistent is set to True because the tape is used more than
    # once to calculate the gradients.
    with tf.GradientTape(persistent=True) as tape:
        # Y = W
        # Generator W translates N -> W
        # Generator N translates W -> N.
        fake_w = generator_w(real_n, training=True)
        cycled_n = generator_n(fake_w, training=True)
        #
        fake_n = generator_n(real_w, training=True)
        cycled_w = generator_w(fake_n, training=True)
        #
        # same_n and same_w are used for identity loss.
        same_n = generator_n(real_n, training=True)
        same_w = generator_w(real_w, training=True)
        #
        disc_real_n = discriminator_n(real_n, training=True)
        disc_real_w = discriminator_w(real_w, training=True)
        #
        disc_fake_n = discriminator_n(fake_n, training=True)
        disc_fake_w = discriminator_w(fake_w, training=True)
        #
        # calculate the loss
        gen_w_loss = generator_loss(disc_fake_w)
        gen_n_loss = generator_loss(disc_fake_n)
        #
        total_cycle_loss = calc_cycle_loss(real_n, cycled_n) + calc_cycle_loss(real_w, cycled_w)
        #
        # Total generator loss = adversarial loss + cycle loss
        total_gen_w_loss = gen_w_loss + total_cycle_loss + identity_loss(real_w, same_w)
        total_gen_n_loss = gen_n_loss + total_cycle_loss + identity_loss(real_n, same_n)
        #
        disc_n_loss = discriminator_loss(disc_real_n, disc_fake_n)
        disc_w_loss = discriminator_loss(disc_real_w, disc_fake_w)
    # Calculate the gradients for generator and discriminator
    generator_w_gradients = tape.gradient(total_gen_w_loss, generator_w.trainable_variables)
    generator_n_gradients = tape.gradient(total_gen_n_loss, generator_n.trainable_variables)
    #
    discriminator_n_gradients = tape.gradient(disc_n_loss, discriminator_n.trainable_variables)
    discriminator_w_gradients = tape.gradient(disc_w_loss, discriminator_w.trainable_variables)
    #
    # Apply the gradients to the optimizer
    generator_w_optimizer.apply_gradients(zip(generator_w_gradients, generator_w.trainable_variables))
    #
    generator_n_optimizer.apply_gradients(zip(generator_n_gradients, generator_n.trainable_variables))
    #
    discriminator_n_optimizer.apply_gradients(zip(discriminator_n_gradients,discriminator_n.trainable_variables))
    #
    discriminator_w_optimizer.apply_gradients(zip(discriminator_w_gradients,discriminator_w.trainable_variables))

In [ ]:
EPOCHS = 50

for epoch in tqdm(range(EPOCHS)):
    start = time.time()
    n = 0
    for image_n, image_w in tf.data.Dataset.zip((train_normal, train_watermark)):
        train_step(image_n[0], image_w[0])
        if n % 10 == 0:
              print ('.', end='')
        n+=1
    clear_output(wait=True)
    # Using a consistent image (sample_watermark) so that the progress of the model
    # is clearly visible.
    generate_images(generator_n, sample_watermark[0])
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1, time.time()-start))
  


## Save Models <a id="save-models"></a>

In [ ]:
generator_w.save(model_dir / "generator_w")
model_dir / "generator_w"

In [ ]:
discriminator_n.save(model_dir / "discriminator_n")
model_dir / "discriminator_n"

In [ ]:
generator_n.save(model_dir / "generator_n")
model_dir / "generator_n"

In [ ]:
discriminator_w.save(model_dir / "discriminator_w")
model_dir / "discriminator_w"